# **MODÈLE PRÉDICTIF POUR ACCORD DE PRÊT : Nettoyage des données.**

## **Check-up des données**

#### Analyse de forme :

- **Variable Cible** : 'MIS_Status'

- **Lignes et colonnes** : (899164, 27)

- **Noms des variables** :

    - LoanNr_ChkDgt
    - Name
    - City
    - State
    - Zip
    - Bank
    - BankState
    - NAICS
    - ApprovalDate
    - ApprovalFY
    - Term
    - NoEmp
    - NewExist,
    - CreateJob
    - RetainedJob
    - FranchiseCode
    - UrbanRural
    - RevLineCr
    - LowDoc
    - ChgOffDate
    - DisbursementDate
    - DisbursementGross
    - BalanceGross
    - MIS_Status
    - ChgOffPrinGr
    - GrAppv
    - SBA_Appv

- **Types de variables** :

    - Qualitatives : 17 string

    - Quantitatives : 9 integer, 1 float


- **Analyse de valeurs manquantes** : Nous avons peu de données manquantes dans notre dataset.

- **Analyse des doublons** : Il y a 147 doublons.

#### Analyse de fond :

- **Visualisation de la target** : Les données de notre variable cible ne sont pas équitablement réparties, nous avons 17% des entreprises qui ont été radiées contre 83% de paiements terminés.

- **Visualisation des variables** : 
    - Les données du dataset vont jusqu'en 2010 et nous avons des données en 2060+, ce sont donc des erreurs qui ont été retiré du dataset. Nous avons également décidé de retirer toutes les données supérieur à 2007, car beaucoup d'entreprises ont été radiés à la suite de la crise économique et cela risque de corrompre les futures prédictions du modèle.

    - La colonne 'LowDoc' ne comprends normalement que 2 valeurs : Y ou N. Or nous avons trouvés d'autres valeurs qui n'étaient pas censées être présentes. La grande majorité de nos données étaient sur "N", c'est pourquoi nous avons préférés remplacer les erreurs par "N" plutôt que supprimer les données.

    - Le code NAIC permet d'identifier le secteur d'activité des différentes entreprises, celui est identifiable par les 2 premiers chiffres du code, c'est pourquoi nous avons remplacé le code par un secteur composé des 2 premiers chiffres.

- **Relation Variables / Target** :

- **Après nettoyage** : Il nous reste 745451 lignes × 10 colonnes (ville, pays, nb_employes, creation, emploi_crees, emplois_retenus, emplacement, petit_pret, secteur, TARGET)

- **Encodage TARGET** : Si la target est égal à 1 : l'entreprise a été radié, si 0 : elle a fini de payé son crédit.

## **Imports et Dataframes**

### Imports

In [2]:
import pandas as pd
import numpy as np

from unidecode import unidecode as uni
import datetime as dt
import re

import matplotlib.pyplot as plt
import seaborn as sns

### Dataframes

In [3]:
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)

In [4]:
data = pd.read_csv('données/SBAnational.csv', low_memory=False)
variables_data = pd.read_csv('données/description_var.csv', low_memory=False)

In [5]:
df = data.copy()
variables = variables_data.copy()

In [6]:
df

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,84,4,2.0,0,0,1,0,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,60,2,2.0,0,0,1,0,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,180,7,1.0,0,0,1,0,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,60,2,1.0,0,0,1,0,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,240,14,1.0,7,7,1,0,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,9995573004,FABRIC FARMS,UPPER ARLINGTON,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,451120,27-Feb-97,1997,60,6,1.0,0,0,1,0,0,N,NaN,30-Sep-97,"$70,000.00",$0.00,P I F,$0.00,"$70,000.00","$56,000.00"
899160,9995603000,FABRIC FARMS,COLUMBUS,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,451130,27-Feb-97,1997,60,6,1.0,0,0,1,0,Y,N,NaN,31-Oct-97,"$85,000.00",$0.00,P I F,$0.00,"$85,000.00","$42,500.00"
899161,9995613003,"RADCO MANUFACTURING CO.,INC.",SANTA MARIA,CA,93455,"RABOBANK, NATIONAL ASSOCIATION",CA,332321,27-Feb-97,1997,108,26,1.0,0,0,1,0,N,N,NaN,30-Sep-97,"$300,000.00",$0.00,P I F,$0.00,"$300,000.00","$225,000.00"
899162,9995973006,"MARUTAMA HAWAII, INC.",HONOLULU,HI,96830,BANK OF HAWAII,HI,0,27-Feb-97,1997,60,6,1.0,0,0,1,0,N,Y,8-Mar-00,31-Mar-97,"$75,000.00",$0.00,CHGOFF,"$46,383.00","$75,000.00","$60,000.00"


### Traduction

In [7]:
variables['Traduction'] = ''
variables['Traduction'].iloc[0] = "Index"
variables['Traduction'].iloc[1] = "Nom de l'emprunteur"
variables['Traduction'].iloc[2] = "Ville de l'emprunteur"
variables['Traduction'].iloc[3] = "Pays de l'emprunteur"
variables['Traduction'].iloc[4] = "Code postal de l'emprunteur"
variables['Traduction'].iloc[5] = "Nom de la banque"
variables['Traduction'].iloc[6] = "Pays de la banque"
variables['Traduction'].iloc[7] = "Code du système de classification des industries nord-américaines"
variables['Traduction'].iloc[8] = "Date de l'engagement de SBA"
variables['Traduction'].iloc[9] = "Année fiscale d'engagement"
variables['Traduction'].iloc[10] = "Durée du prêt en mois"
variables['Traduction'].iloc[11] = "Nombre d'employés"
variables['Traduction'].iloc[12] = "1 : Entreprise existante, 2 : Nouvelle entreprise"
variables['Traduction'].iloc[13] = "Nombre d'emplois créés"
variables['Traduction'].iloc[14] = "Nombre d'emplois conservés"
variables['Traduction'].iloc[15] = "Code franchise, (00000 or 00001) = Pas de franchise"
variables['Traduction'].iloc[16] = "1 = Ville, 2 = Campagne, 0 = Indéfini"
variables['Traduction'].iloc[17] = "Ligne de crédit renouvelable :  Y = Yes, N = No"
variables['Traduction'].iloc[18] = "Programme de prêt LowDoc : O = Oui, N = Non"
variables['Traduction'].iloc[19] = "Date prêt en défaut de paiement"
variables['Traduction'].iloc[20] = "Date de décaissement"
variables['Traduction'].iloc[21] = "Montant du décaissement"
variables['Traduction'].iloc[22] = "Montant brut en circulation"
variables['Traduction'].iloc[23] = "Radié = CHGOFF, Payé en totalité = PIF"
variables['Traduction'].iloc[24] = "Montant imputé"
variables['Traduction'].iloc[25] = "Montant brut du prêt approuvé par la banque"
variables['Traduction'].iloc[26] = "Montant garanti par la SBA pour le prêt approuvé"

del variables['Description']

variables

,Variable Name,Traduction
0,LoanNr_ChkDgt,Index
1,Name,Nom de l'emprunteur
2,City,Ville de l'emprunteur
3,State,Pays de l'emprunteur
4,Zip,Code postal de l'emprunteur
5,Bank,Nom de la banque
6,BankState,Pays de la banque
7,NAICS,Code du système de classification des industri...
8,ApprovalDate,Date de l'engagement de SBA
9,ApprovalFY,Année fiscale d'engagement


In [8]:
variables.columns

Index(['Variable Name', 'Traduction'], dtype='object')

### Infos

In [9]:
df['MIS_Status'].value_counts()

P I F     739609
CHGOFF    157558
Name: MIS_Status, dtype: int64

In [10]:
df.shape

(899164, 27)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      899164 non-null  int64  
 1   Name               899150 non-null  object 
 2   City               899134 non-null  object 
 3   State              899150 non-null  object 
 4   Zip                899164 non-null  int64  
 5   Bank               897605 non-null  object 
 6   BankState          897598 non-null  object 
 7   NAICS              899164 non-null  int64  
 8   ApprovalDate       899164 non-null  object 
 9   ApprovalFY         899164 non-null  object 
 10  Term               899164 non-null  int64  
 11  NoEmp              899164 non-null  int64  
 12  NewExist           899028 non-null  float64
 13  CreateJob          899164 non-null  int64  
 14  RetainedJob        899164 non-null  int64  
 15  FranchiseCode      899164 non-null  int64  
 16  Ur

In [12]:
df.dtypes.value_counts()

object     17
int64       9
float64     1
dtype: int64

In [13]:
df.columns

Index(['LoanNr_ChkDgt', 'Name', 'City', 'State', 'Zip', 'Bank', 'BankState',
       'NAICS', 'ApprovalDate', 'ApprovalFY', 'Term', 'NoEmp', 'NewExist',
       'CreateJob', 'RetainedJob', 'FranchiseCode', 'UrbanRural', 'RevLineCr',
       'LowDoc', 'ChgOffDate', 'DisbursementDate', 'DisbursementGross',
       'BalanceGross', 'MIS_Status', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv'],
      dtype='object')

In [14]:
df['FranchiseCode'].value_counts()

1        638554
0        208835
78760      3373
68020      1921
50564      1034
          ...  
24421         1
61615         1
81580         1
83876         1
15930         1
Name: FranchiseCode, Length: 2768, dtype: int64

### To_datetime

In [13]:
df['ApprovalDate'] = pd.to_datetime(df['ApprovalDate'])
df['ApprovalDate'] = df['ApprovalDate'].dt.year

### Unicode

In [14]:
df['Name'] = df['Name'].apply(lambda x : uni(str(x).upper()))
df['City'] = df['City'].apply(lambda x : uni(str(x).upper()))
df['State'] = df['State'].apply(lambda x : uni(str(x).upper()))
df['Bank'] = df['Bank'].apply(lambda x : uni(str(x).upper()))
df['BankState'] = df['BankState'].apply(lambda x : uni(str(x).upper()))
del df['LoanNr_ChkDgt']

In [15]:
df['SBA_Appv'].value_counts()

$25,000.00      49579
$12,500.00      40147
$5,000.00       31135
$50,000.00      25047
$10,000.00      17009
                ...  
$192,949.00         1
$212,352.00         1
$57,072.00          1
$32,938.00          1
$715,674.00         1
Name: SBA_Appv, Length: 38326, dtype: int64

## **Nettoyage des données**

### Valeurs manquantes

In [16]:
# plt.figure(figsize=(20,10))
# sns.heatmap(df.isna(), cbar=False)
# plt.show()

In [17]:
(df.isna().sum() / df.shape[0]).sort_values(ascending=False)

ChgOffDate           0.819055
RevLineCr            0.005036
LowDoc               0.002872
DisbursementDate     0.002634
MIS_Status           0.002221
NewExist             0.000151
Name                 0.000000
FranchiseCode        0.000000
GrAppv               0.000000
ChgOffPrinGr         0.000000
BalanceGross         0.000000
DisbursementGross    0.000000
UrbanRural           0.000000
RetainedJob          0.000000
City                 0.000000
CreateJob            0.000000
NoEmp                0.000000
Term                 0.000000
ApprovalFY           0.000000
ApprovalDate         0.000000
NAICS                0.000000
BankState            0.000000
Bank                 0.000000
Zip                  0.000000
State                0.000000
SBA_Appv             0.000000
dtype: float64

In [18]:
df.drop(df[df['State'] == 'NAN'].index, axis=0, inplace=True)

In [19]:
df['RevLineCr'] = df['RevLineCr'].apply(lambda x : 'Y' if x == 'Y' or x == 'T' else 'N')

In [20]:
df['RevLineCr'].value_counts()

N    682469
Y    216681
Name: RevLineCr, dtype: int64

In [21]:
df[df['GrAppv'] < df['DisbursementGross']]

,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
28,WEYLAND CORPORATION,CAMARILLO,CA,93010,WELLS FARGO BANK NATL ASSOC,SD,611110,2006,2006,83,18,2.0,5,23,1,1,Y,N,NaN,28-Feb-06,"$438,541.00",$0.00,P I F,$0.00,"$100,000.00","$50,000.00"
30,CHICAGO BRICK UNLIMITED INC,MIAMI,FL,33186,"CITIBANK, N.A.",FL,238140,2006,2006,84,4,1.0,0,4,1,1,Y,N,NaN,28-Feb-06,"$51,440.00",$0.00,P I F,$0.00,"$35,000.00","$17,500.00"
34,HUTMACHER HOLDINGS INC,LEANDER,TX,78641,WELLS FARGO BANK NATL ASSOC,SD,541611,2006,2006,80,2,1.0,4,6,1,2,Y,N,NaN,31-May-06,"$63,076.00",$0.00,P I F,$0.00,"$25,000.00","$12,500.00"
48,ANYWHERE SPORTS PRODUCTIONS,MARINA DEL RAY,CA,90292,WELLS FARGO BANK NATL ASSOC,SD,0,1997,1997,84,6,1.0,0,0,1,0,Y,N,NaN,31-Oct-97,"$30,570.00",$0.00,P I F,$0.00,"$25,000.00","$12,500.00"
65,B&B DIPNETS,WASILLA,AK,99654,WELLS FARGO BANK NATL ASSOC,SD,314999,2006,2006,83,2,1.0,0,2,1,1,Y,N,NaN,28-Feb-06,"$78,218.00",$0.00,P I F,$0.00,"$30,000.00","$15,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898580,"FALKENBERG, INC.",CLACKAMAS,OR,97015,JPMORGAN CHASE BANK NATL ASSOC,OR,0,1997,1997,12,12,1.0,0,0,1,0,Y,N,NaN,30-Apr-97,"$311,192.00",$0.00,P I F,$0.00,"$200,000.00","$150,000.00"
898593,CARING HANDS ANIMAL HOSPITAL,FRANKLIN,TN,37064,FIFTH THIRD BANK,TN,541940,1997,1997,120,5,2.0,0,0,1,0,N,Y,NaN,31-Mar-97,"$99,992.00",$0.00,P I F,$0.00,"$100,000.00","$80,000.00"
898624,VICTORIAN MANOR RESTAURANT,GENESEO,IL,61254,WELLS FARGO BANK NATL ASSOC,SD,0,1997,1997,60,22,1.0,0,0,1,0,Y,N,NaN,31-Jul-97,"$25,297.00",$0.00,P I F,$0.00,"$25,000.00","$12,500.00"
898679,"MINORITY TEMPORARY AGENCY, INC",NEW YORK,NY,10017,"C & A INVESTMENTS, INC",NY,0,1997,1997,12,6,1.0,0,0,1,0,Y,N,NaN,30-Jun-97,"$418,930.00",$0.00,P I F,$0.00,"$100,000.00","$50,000.00"


### Doublons

In [22]:
test = df[df.duplicated()].index

df.drop(test, axis=0, inplace=True)

### Première sélection de colonnes

In [23]:
variables[variables['Variable Name'] == "NewExist"]

,Variable Name,Traduction
12,NewExist,"1 : Entreprise existante, 2 : Nouvelle entreprise"


In [24]:
df = df.drop(['ChgOffDate','Bank', 'BankState','City', 'Name','Zip', 'ApprovalFY','RetainedJob', 'SBA_Appv', 'DisbursementDate', 'DisbursementGross', 'BalanceGross', 'ChgOffPrinGr'], axis=1).dropna()

df

,State,NAICS,ApprovalDate,Term,NoEmp,NewExist,CreateJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,GrAppv
0,IN,451120,1997,84,4,2.0,0,1,0,N,Y,P I F,"$60,000.00"
1,IN,722410,1997,60,2,2.0,0,1,0,N,Y,P I F,"$40,000.00"
2,IN,621210,1997,180,7,1.0,0,1,0,N,N,P I F,"$287,000.00"
3,OK,0,1997,60,2,1.0,0,1,0,N,Y,P I F,"$35,000.00"
4,FL,0,1997,240,14,1.0,7,1,0,N,N,P I F,"$229,000.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,OH,451120,1997,60,6,1.0,0,1,0,N,N,P I F,"$70,000.00"
899160,OH,451130,1997,60,6,1.0,0,1,0,Y,N,P I F,"$85,000.00"
899161,CA,332321,1997,108,26,1.0,0,1,0,N,N,P I F,"$300,000.00"
899162,HI,0,1997,60,6,1.0,0,1,0,N,Y,CHGOFF,"$75,000.00"


In [25]:
df.shape

(894333, 13)

### Nettoyage des données

In [26]:
def secteur (naics):
    secteur = str(naics)[:2]
    secteur = int(secteur)
    if secteur == 32 or secteur == 33:
        secteur = 31

    elif secteur == 45:
        secteur = 44

    elif secteur == 49:
        secteur = 48

    return secteur

In [27]:
df['secteur'] = df['NAICS'].apply(lambda x : secteur(x))
del df['NAICS']

In [28]:
test = df[df['NewExist'] == 0.0].index

df.drop(test,axis=0, inplace=True)

df['NewExist'] = df['NewExist'].apply(lambda x : 1 if x == 2 else 0)

In [29]:
df['NewExist'] = df['NewExist'].apply(lambda x : int(x))

In [30]:
df['LowDoc'] = df['LowDoc'].apply(lambda x : "N" if x != "Y" else "Y")

In [31]:
test = df[df['ApprovalDate'] >= 2007].index

df.drop(test,axis=0, inplace=True)

del df['ApprovalDate']

In [32]:
df.rename(columns={'State' : 'pays','ApprovalDate':'date','FranchiseCode' : 'franchise','GrAppv' : 'montant', 'NoEmp' : 'nb_employes','RevLineCr' : 'cred_renouvelable','Term' : 'mois', 'NewExist' : 'creation', 'CreateJob' : 'emploi_crees', 'RetainedJob' : 'emplois_retenus', 'UrbanRural' : 'emplacement','LowDoc' : 'petit_pret'}, inplace=True)

In [33]:
df.columns

Index(['pays', 'mois', 'nb_employes', 'creation', 'emploi_crees', 'franchise',
       'emplacement', 'cred_renouvelable', 'petit_pret', 'MIS_Status',
       'montant', 'secteur'],
      dtype='object')

### Encodage de la target

In [34]:
df['TARGET'] = df['MIS_Status'].apply(lambda x : 1 if x == 'CHGOFF' else 0)
del df['MIS_Status']

## **Analyses préléminaires**

In [35]:
df

,pays,mois,nb_employes,creation,emploi_crees,franchise,emplacement,cred_renouvelable,petit_pret,montant,secteur,TARGET
0,IN,84,4,1,0,1,0,N,Y,"$60,000.00",44,0
1,IN,60,2,1,0,1,0,N,Y,"$40,000.00",72,0
2,IN,180,7,0,0,1,0,N,N,"$287,000.00",62,0
3,OK,60,2,0,0,1,0,N,Y,"$35,000.00",0,0
4,FL,240,14,0,7,1,0,N,N,"$229,000.00",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
899159,OH,60,6,0,0,1,0,N,N,"$70,000.00",44,0
899160,OH,60,6,0,0,1,0,Y,N,"$85,000.00",44,0
899161,CA,108,26,0,0,1,0,N,N,"$300,000.00",31,0
899162,HI,60,6,0,0,1,0,N,Y,"$75,000.00",0,1


In [36]:
df.dtypes

pays                 object
mois                  int64
nb_employes           int64
creation              int64
emploi_crees          int64
franchise             int64
emplacement           int64
cred_renouvelable    object
petit_pret           object
montant              object
secteur               int64
TARGET                int64
dtype: object

In [37]:
df['TARGET'].value_counts(normalize=True)

0    0.852461
1    0.147539
Name: TARGET, dtype: float64

In [38]:
# plt.figure()
# sns.pairplot(df)
# plt.savefig("graphiques/first/pairplot_clean.png")

In [39]:
# for col in df.select_dtypes('int'):
#     plt.figure()
#     sns.histplot(df[col], common_norm=True)
#     plt.savefig(f"graphiques/first/hist_{col}.png")
#     plt.show()

In [40]:
for col in df.select_dtypes('object'):
    print(f"{col :-<20} {df[col].unique()}")

pays---------------- ['IN' 'OK' 'FL' 'CT' 'NJ' 'NC' 'IL' 'RI' 'TX' 'VA' 'TN' 'AR' 'MN' 'MO'
 'MA' 'CA' 'SC' 'LA' 'IA' 'OH' 'KY' 'MS' 'NY' 'MD' 'PA' 'OR' 'ME' 'KS'
 'MI' 'AK' 'WA' 'CO' 'MT' 'WY' 'UT' 'NH' 'WV' 'ID' 'AZ' 'NV' 'WI' 'NM'
 'GA' 'ND' 'VT' 'AL' 'NE' 'SD' 'HI' 'DE' 'DC']
cred_renouvelable--- ['N' 'Y']
petit_pret---------- ['Y' 'N']
montant------------- ['$60,000.00 ' '$40,000.00 ' '$287,000.00 ' ... '$12,480.00 '
 '$62,425.00 ' '$1,086,300.00 ']


In [41]:
# for col in df.select_dtypes('object'):
#     plt.figure()
#     df[col].value_counts().plot(kind='bar',title=f"Répartition de la variable '{col}'")
#     plt.savefig(f"graphiques/first/bar_{col}.png")
#     plt.show()

In [42]:
df.shape

(745450, 12)

In [43]:
df.to_csv('données/données_clean.csv', index=False)